In [1]:
pwd

'/gpfs/alpine/bif135/proj-shared/structural_DLFA_data/P_mercurii/Annotation_work'

In [2]:
import sys
sys.path.insert(0,'/ccs/home/davidsonrb/Scripts/git/structural_DLFA/parsers/')

In [3]:
import apoc_parsers
import rcsb_query

In [20]:
from pathlib import Path
import numpy as np
import pandas as pd
import gzip
import requests
from urllib import request
import datetime

In [36]:
def uniprot_search(uniprotID,species_mnemonic=None):
    '''code to access UniprotKB RESTful API metadata file for a uniprotID
    pull _some_ of the relevant metadata and store in a dictionary 
    '''
    metadata_url = 'https://www.uniprot.org/uniprot/%s.txt'
    dictionary_space = {'uniprotID': uniprotID,  # 
                        'entry_name': '',        # ID line
                        'status': '',            # ID line
                        'sequence_length': '',   # ID line
                        'original_date': '',     # 1st DT line
                        'sequence_date': '',     # 2nd DT line
                        'modified_date': '',     # 3rd DT line
                        'gene_name': '',         # GN line
                        'species_name': '',      # OS line
                        'database_references': [], # DR lines
                        'primary_evidence': '',  # PE line
                        'features': [],          # FT lines
                        'sequence': '>',          # SQ and blank lines afterwards...
                        'ecIDs': [],
                        'success': 0
                        }
    
    try:
        response = request.urlopen(metadata_url%(uniprotID))
        response_str = str(response.read())
        response_lines = response_str[2:-1].split('\\n')
        for line in response_lines:
            if 'EC=' in line:
                dictionary_space['ecIDs'].append(line)
            
            if line[:2] == 'ID':
                temp = line.split()
                if species_mnemonic and species_mnemonic not in temp[1]:
                    #log_file_handle.write(f'This uniprotID ({uniprotID}) does not originate from the expected species ({species_mnemonic}). Killing task.\n')
                    print(f'This uniprotID ({uniprotID}) does not originate from the expected species ({species_mnemonic}). Killing task.\n')
                    break  # pops us out of the loop and returns an empty dictionary space with a 'success' value of 0...
                
                dictionary_space['success'] = 1
                dictionary_space['entry_name'] = temp[1]
                dictionary_space['status']     = temp[2][:-1]           # there's a stupid semicolon at the end of the status 
                dictionary_space['sequence_length'] = int(temp[3])
            elif line[:2] == 'DT':
                if 'integrated' in line:
                    dictionary_space['original_date'] = datetime.datetime.strptime(line.split()[1][:-1],'%d-%b-%Y').strftime('%Y-%m-%d')
                elif 'sequence version' in line:
                    dictionary_space['sequence_date'] = datetime.datetime.strptime(line.split()[1][:-1],'%d-%b-%Y').strftime('%Y-%m-%d')
                elif 'entry version' in line:
                    dictionary_space['modified_date'] = datetime.datetime.strptime(line.split()[1][:-1],'%d-%b-%Y').strftime('%Y-%m-%d')
            elif line[:2] == 'GN':
                dictionary_space['gene_name'] += line[5:]
            elif line[:2] == 'OS':
                dictionary_space['species_name'] += line[5:]
            elif line[:2] == 'DR':
                dictionary_space['database_references'].append([line[5:]])
            elif line[:2] == 'PE':
                dictionary_space['primary_evidence'] += line[5:]
            elif line[:2] == 'FT' and line[5] != ' ':
                dictionary_space['features'].append(line[5:].split())
            elif line[:2] == 'FT' and line[5] == ' ':
                dictionary_space['features'][-1].append(line.split()[-1])
                
            elif line[:2] == 'SQ' and line[5] != ' ':
                dictionary_space['sequence'] += line[5:] + '\n'
            elif line[:2] == '  ':
                temp = line.split()
                seq_string = ''.join(temp)
                dictionary_space['sequence'] += seq_string
        return dictionary_space
    
    except Exception as e:
        e = str(e)
        if '400' in e:
            print(uniprotID, e, 'Bad request. There is a problem with your input.')
        elif '404' in e:
            print(uniprotID, e, "Not found. The resource you requested doesn't exist.")
        elif '410' in e:
            print(uniprotID, e, 'Gone. The resource you requested was removed.')
        elif '500' in e:
            print(uniprotID, e, 'Internal server error. Most likely a temporary problem, but if the problem persists please contact us.')
        elif '503' in e:
            print(uniprotID, e, 'Service not available. The server is being updated, try again later.')
        else:
            print(uniprotID, e, 'something else funky is going on...')
        return dictionary_space


# Gather APoc results

In [5]:
test = Path('/gpfs/alpine/proj-shared/bif135/structural_DLFA_data/P_mercurii/TMalign-APoc/WP_014320990.1/WP_014320990.1_sco.dat.gz')
test_scores = apoc_parsers.parse_apoc_score_file(str(test))
test_scores

,protein,tname,malnlen,mrmsd,mscore,mseqid,Description
0,WP_014320990.1,5T0W_C,228,1.59,0.75768,48.7,"AncCDT-1; solute-binding protein, periplasmic ..."
1,WP_014320990.1,4Z9N_B,238,2.39,0.75031,17.6,"Amino acid ABC transporter, periplasmic; ABC t..."
2,WP_014320990.1,4Z9N_F,236,2.36,0.74369,18.2,"Amino acid ABC transporter, periplasmic; ABC t..."
3,WP_014320990.1,4H5F_C,234,2.23,0.74271,21.4,Amino acid ABC superfamily ATP; CENTER FOR STR...
4,WP_014320990.1,3K4U_E,226,1.77,0.73989,38.5,BINDING COMPONENT OF ABC TRANSPORTER; STRUCTUR...
...,...,...,...,...,...,...,...
994,WP_014320990.1,4MPT_A,190,5.21,0.44296,5.8,Putative leu/ile/val-binding protein; Structur...
995,WP_014320990.1,6HNI_A,186,5.31,0.44289,5.4,"ABC-type transport system, sugar-family extrac..."
996,WP_014320990.1,3UK0_A,188,5.25,0.44288,4.8,Extracellular ligand-binding receptor; STRUCTU...
997,WP_014320990.1,3CS3_A,179,4.90,0.44224,8.4,"Sugar-binding transcriptional regulator, LacI ..."


In [6]:
with gzip.open(str(test),mode='rt') as f:
    lines = f.readlines()
print(lines[:10])

protein = apoc_parsers._get_protein(str(test))
print(protein)

temp = lines[2].strip().split('|')
print(temp)

rec = [protein] + temp[0].strip().split() + [temp[1].strip()]
print(rec)

['tname  malnlen  mrmsd   mscore  mseqid\tDescription\n', '5T0W_C     228  1.590 0.75768  48.7%\t| AncCDT-1; solute-binding protein, periplasmic binding protein; HET: ARG; 2.59A {synthetic construct}\n', '4Z9N_B     238  2.390 0.75031  17.6%\t| Amino acid ABC transporter, periplasmic; ABC transporter, glutathione, GSH, Structural; HET: EDO, GSH; 1.745A {Brucella ovis (strain ATCC 25840 / 63/290 / NCTC 10512)}\n', '4Z9N_F     236  2.360 0.74369  18.2%\t| Amino acid ABC transporter, periplasmic; ABC transporter, glutathione, GSH, Structural; HET: EDO, GSH; 1.745A {Brucella ovis (strain ATCC 25840 / 63/290 / NCTC 10512)}\n', '4H5F_C     234  2.230 0.74271  21.4%\t| Amino acid ABC superfamily ATP; CENTER FOR STRUCTURAL GENOMICS OF; HET: ARG, SO4, PEG, ACT, GOL, PGE; 1.9A {Streptococcus pneumoniae}\n', '3K4U_E     226  1.770 0.73989  38.5%\t| BINDING COMPONENT OF ABC TRANSPORTER; STRUCTURAL GENOMICS, CRYSTAL STRUCTURE,PROTEIN STRUCTURE; HET: LYS; 2.62A {Wolinella succinogenes}\n', '1XT8_B  

In [7]:
test = Path('/gpfs/alpine/proj-shared/bif135/structural_DLFA_data/P_mercurii/TMalign-APoc/WP_014320990.1/WP_014320990.1_aln.dat.gz')
test_alns = apoc_parsers.parse_apoc_aln_file(str(test))
test_alns

,protein,Aln_num,Prot_ID,naln,score,sid,Ind,Ch1,Res1,AA1,Ch2,Res2,AA2,Dist,Cos,S
0,WP_014320990.1,1,5T0W_C,228,0.7577,48.7,1,C,14,S,A,46,S,0.913,0.995,1.0
1,WP_014320990.1,1,5T0W_C,228,0.7577,48.7,2,C,15,T,A,47,Q,0.662,0.997,0.0
2,WP_014320990.1,1,5T0W_C,228,0.7577,48.7,3,C,16,L,A,48,L,0.428,0.998,1.0
3,WP_014320990.1,1,5T0W_C,228,0.7577,48.7,4,C,17,D,A,49,D,0.601,0.993,1.0
4,WP_014320990.1,1,5T0W_C,228,0.7577,48.7,5,C,18,E,A,50,V,0.747,1.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2326,WP_014320990.1,10,5EYF_B,227,0.7346,25.6,223,B,261,W,A,275,W,0.345,0.976,1.0
2327,WP_014320990.1,10,5EYF_B,227,0.7346,25.6,224,B,262,F,A,276,F,1.940,0.984,1.0
2328,WP_014320990.1,10,5EYF_B,227,0.7346,25.6,225,B,263,P,A,277,F,2.637,-0.602,0.0
2329,WP_014320990.1,10,5EYF_B,227,0.7346,25.6,226,B,264,N,A,278,T,5.473,-0.474,0.0


# Gather UNIPROT IDs from Hits

In [8]:
test_scores['tname']

0      5T0W_C
1      4Z9N_B
2      4Z9N_F
3      4H5F_C
4      3K4U_E
        ...  
994    4MPT_A
995    6HNI_A
996    3UK0_A
997    3CS3_A
998    4RXU_A
Name: tname, Length: 999, dtype: string

In [9]:
uniprotids = []
interesting_uniprotids = []
for hit in test_scores.iloc():
    if hit['mscore'] > 0.60:
        pdbid_chainid = hit['tname']
        uniprotid = rcsb_query.query_uniprot_str(pdbid_chainid)
        uniprotids.append(uniprotid)
        if uniprotid != None:
            #print(pdbid_chainid,uniprotid,hit['mscore'],hit['mrmsd'],hit['mseqid'],hit['Description'][:10])
            interesting_uniprotids.append(uniprotid)
    else:
        uniprotids.append('N/A')

print(interesting_uniprotids[:10],len(interesting_uniprotids))
interesting_uniprotids = list(set(interesting_uniprotids))
print(interesting_uniprotids[:10],len(interesting_uniprotids))

['A0A0M3KL33', 'A0A0M3KL33', 'Q7MAG0', 'Q0P9S0', 'Q9HXN1', 'Q9HXN1', 'Q3XZW5', 'Q72JG5', 'A0A0F4FYR3', 'A0A0H2ZN67'] 141
['A0A0H2UKY8', 'O69061', 'P02911', 'Q8ZKA9', 'P96257', 'Q9VDH5', 'P42400', 'P23819', 'Q3XZW5', 'Q93DA5'] 79


In [10]:
test_scores.insert(2,'UniProtID',uniprotids,True)
#test_scores['UniProtIDs'] = uniprotids
print(test_scores)

            protein   tname   UniProtID  malnlen  mrmsd   mscore  mseqid  \
0    WP_014320990.1  5T0W_C        None      228   1.59  0.75768    48.7   
1    WP_014320990.1  4Z9N_B  A0A0M3KL33      238   2.39  0.75031    17.6   
2    WP_014320990.1  4Z9N_F  A0A0M3KL33      236   2.36  0.74369    18.2   
3    WP_014320990.1  4H5F_C        None      234   2.23  0.74271    21.4   
4    WP_014320990.1  3K4U_E      Q7MAG0      226   1.77  0.73989    38.5   
..              ...     ...         ...      ...    ...      ...     ...   
994  WP_014320990.1  4MPT_A         N/A      190   5.21  0.44296     5.8   
995  WP_014320990.1  6HNI_A         N/A      186   5.31  0.44289     5.4   
996  WP_014320990.1  3UK0_A         N/A      188   5.25  0.44288     4.8   
997  WP_014320990.1  3CS3_A         N/A      179   4.90  0.44224     8.4   
998  WP_014320990.1  4RXU_A         N/A      194   5.64  0.44214     7.7   

                                           Description  
0    AncCDT-1; solute-binding 

In [37]:
test = uniprot_search('A0A0M3KL33')
test

{'uniprotID': 'A0A0M3KL33',
 'entry_name': 'A0A0M3KL33_BRUO2',
 'status': 'Unreviewed',
 'sequence_length': 214,
 'original_date': '2015-11-11',
 'sequence_date': '2015-11-11',
 'modified_date': '2022-08-03',
 'gene_name': 'ORFNames=BOV_0736 {ECO:0000313|PDB:4Z9N};',
 'species_name': 'Brucella ovis (strain ATCC 25840 / 63/290 / NCTC 10512).',
 'database_references': [['PDB; 4Z9N; X-ray; 1.75 A; A/B/C/D/E/F=1-214.'],
  ['PDBsum; 4Z9N; -.'],
  ['SMR; A0A0M3KL33; -.'],
  ['InterPro; IPR001638; Solute-binding_3/MltF_N.'],
  ['Pfam; PF00497; SBP_bac_3; 1.']],
 'primary_evidence': '1: Evidence at protein level;',
 'features': [['DOMAIN',
   '5..124',
   '/note="PBPb"',
   '/evidence="ECO:0000259|Pfam:PF00497"'],
  ['BINDING',
   '24',
   '/ligand="Glutathione"',
   '/ligand_id="ChEBI:CHEBI:16856"',
   '/evidence="ECO:0007829|PDB:4Z9N"'],
  ['BINDING',
   '28',
   '/ligand="Glutathione"',
   '/ligand_id="ChEBI:CHEBI:16856"',
   '/evidence="ECO:0007829|PDB:4Z9N"'],
  ['BINDING',
   '70',
   '/

In [34]:
len(test['sequence'].split('\n')[1])

214